In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import mean_absolute_error
import optuna
from concurrent.futures import ProcessPoolExecutor
from joblib import Parallel, delayed
from scipy.stats import t

In [2]:
Main_data=pd.read_csv('Main_data_case2.csv')
Schedule=pd.read_csv('Schedule.csv')
Origin_result = pd.read_csv('Origin_result.csv')
Main_data.columns

Index(['Team', 'Conference', '2PA', '3PA', 'TOV', 'FTD', 'RAA', 'PZA', 'MRA',
       'COA', 'ABA', '2SFD', '3SFD', 'NSFD', 'RAP', 'PZP', 'MRP', 'COP', 'ABP',
       'FTP', 'PACE', 'DEEPA', 'DEEPP', '2PAnd1', '3PAnd1', 'ORAP', 'OPZP',
       'OMRP', 'OCOP', 'OABP', 'ORB', 'DRB'],
      dtype='object')

In [3]:
Origin_result

,Team,Conference,Win_Percentage,Play_off
0,ATL,East,0.439,1
1,BKN,East,0.390,0
2,BOS,East,0.790,1
3,CHA,East,0.256,0
4,CHI,East,0.476,1
5,CLE,East,0.585,1
6,DAL,West,0.610,1
7,DEN,West,0.695,1
8,DET,East,0.171,0
9,GSW,West,0.561,1


In [7]:
def generate_dirichlet_sample(row,multiply):
    ratios = row[['2PA', '3PA', 'DEEPA', 'FTD', 'TOV']].values
    ratios = ratios.flatten()
    
    alpha_raw = ratios  
    alpha = (alpha_raw / alpha_raw.sum()) * multiply  
    
    dirichlet_sample = np.random.dirichlet(alpha)

    return dirichlet_sample

def generate_dirichlet_sample_2pt(row,multiply):
    ratios = row[['RAA', 'PZA', 'MRA', '2PAnd1']].values
    ratios = ratios.flatten()
    
    alpha_raw = ratios 
    alpha = (alpha_raw / alpha_raw.sum()) * multiply  

    dirichlet_sample_2pt = np.random.dirichlet(alpha)

    return dirichlet_sample_2pt

def generate_dirichlet_sample_3pt(row,multiply):
    ratios = row[['COA', 'ABA', '3PAnd1']].values
    ratios = ratios.flatten()
    
    alpha_raw = ratios  
    alpha = (alpha_raw / alpha_raw.sum()) * multiply  

    dirichlet_sample_3pt = np.random.dirichlet(alpha)

    return dirichlet_sample_3pt

def generate_dirichlet_sample_FT(row,multiply):
    
    ratios = row[['2SFD', '3SFD', 'NSFD']].values
    ratios = ratios.flatten()
    
    alpha_raw = ratios 
    alpha = (alpha_raw / alpha_raw.sum()) * multiply  

    dirichlet_sample_FT = np.random.dirichlet(alpha)

    return dirichlet_sample_FT

In [9]:
def generate_indicator(sample_dist):
        cumulative_probs = np.cumsum(sample_dist)

        random_value = np.random.uniform(0, 1)

        if random_value <= cumulative_probs[0]:
            return '2pt'
        elif random_value <= cumulative_probs[1]:
            return '3pt'
        elif random_value <= cumulative_probs[2]:
            return '4pt'    
        elif random_value <= cumulative_probs[3]:
            return 'FT'
        else:
            return 'TOV'
            
def generate_indicator_2pt(sample_dist):
        cumulative_probs = np.cumsum(sample_dist)

        random_value = np.random.uniform(0, 1)

        if random_value <= cumulative_probs[0]:
            return 'RA'
        elif random_value <= cumulative_probs[1]:
            return 'PZ'
        elif random_value <= cumulative_probs[2]:
            return 'MR'
        else:
            return '2PAnd1'

def generate_indicator_3pt(sample_dist):
        cumulative_probs = np.cumsum(sample_dist)

        random_value = np.random.uniform(0, 1)

        if random_value <= cumulative_probs[0]:
            return 'CO'
        elif random_value <= cumulative_probs[1]:
            return 'AB'
        else:
            return '3PAnd1'

def generate_indicator_FT(sample_dist):
        cumulative_probs = np.cumsum(sample_dist)

        random_value = np.random.uniform(0, 1)

        if random_value <= cumulative_probs[0]:
            return '2SFD'
        elif random_value <= cumulative_probs[1]:
            return '3SFD'
        else:
            return 'NSFD'

In [11]:
def simulation_function(OFF, DEF, rate, multiply, maindata):
    off_team_data = maindata[maindata['Team'] == OFF]
    def_team_data = maindata[maindata['Team'] == DEF]

    if off_team_data.empty or def_team_data.empty:
        raise ValueError("Name_error")

    sample_dist = generate_dirichlet_sample(off_team_data, multiply)
    
    indicator = generate_indicator(sample_dist)

    retry_occurred = False
    nested_result = None  

    if indicator == '2pt':
        sample_dist_step2 = generate_dirichlet_sample_2pt(off_team_data, multiply)
        indicator_step2 = generate_indicator_2pt(sample_dist_step2)
        if indicator_step2 == 'RA':
            success_prob = ((rate * off_team_data['RAP'].iloc[0]) + ((1-rate) * def_team_data['ORAP'].iloc[0])) / 100
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 2
            else:
                point = 0
            if point == 0:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 0

        elif indicator_step2 == 'PZ':
            success_prob = ((rate * off_team_data['PZP'].iloc[0]) + ((1-rate) * def_team_data['OPZP'].iloc[0])) / 100
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 2
            else:
                point = 0
            if point == 0:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 0

        elif indicator_step2 == 'MR':
            success_prob = ((rate * off_team_data['MRP'].iloc[0]) + ((1-rate) * def_team_data['OMRP'].iloc[0])) / 100
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 2
            else:
                point = 0
            if point == 0:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 0

        else:
            success_prob = (off_team_data['FTP'].iloc[0] / 100)
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 3
            else:
                point = 2
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 2
                    
    elif indicator == '3pt':
        sample_dist_step2 = generate_dirichlet_sample_3pt(off_team_data, multiply)
        indicator_step2 = generate_indicator_3pt(sample_dist_step2)
        if indicator_step2 == 'CO':
            success_prob = ((rate * off_team_data['COP'].iloc[0]) + ((1-rate) * def_team_data['OCOP'].iloc[0])) / 100
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 3
            else:
                point = 0
            if point == 0:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 0
        elif indicator_step2 == 'AB':
            success_prob = ((rate * off_team_data['ABP'].iloc[0]) + ((1-rate) * def_team_data['OABP'].iloc[0])) / 100
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 3
            else:
                point = 0
            if point == 0:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 0
            
        else:
            success_prob = (off_team_data['FTP'].iloc[0] / 100)
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 4
            else:
                point = 3
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 3
    elif indicator == '4pt':
        indicator_step2 = '4pt'
        success_prob = off_team_data['DEEPP'].iloc[0] / 100  
        point = 0
        random_value = np.random.uniform(0, 1)
        if random_value <= success_prob:
            point = 4 
        else:
            point = 0
        if point == 0:
            off_orb = off_team_data['ORB'].iloc[0]
            def_drb = def_team_data['DRB'].iloc[0]
            total = off_orb + def_drb
            prob_retry = off_orb / total if total > 0 else 0
            prob_fail = def_drb / total if total > 0 else 1

            retry_prob = np.random.uniform(0, 1)
            if retry_prob <= prob_retry:
                retry_occurred = True
                nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
            else:
                point = 0

    elif indicator == 'FT':
        sample_dist_step2 = generate_dirichlet_sample_FT(off_team_data, multiply)
        indicator_step2 = generate_indicator_FT(sample_dist_step2)
        if indicator_step2 == '2SFD':
            success_prob = off_team_data['FTP'].iloc[0] / 100
            point = 0
            last_shot_success = True
            for _ in range(2):
                random_value = np.random.uniform(0, 1)
                if random_value <= success_prob:
                    point += 1
                    last_shot_success = True
                else:
                    last_shot_success = False

            if not last_shot_success:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = point

        elif indicator_step2 == '3SFD':
            success_prob = off_team_data['FTP'].iloc[0] / 100
            point = 0
            last_shot_success = True
            for _ in range(3):
                random_value = np.random.uniform(0, 1)
                if random_value <= success_prob:
                    point += 1
                    last_shot_success = True
                else:
                    last_shot_success = False

            if not last_shot_success:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = point

        else:
            success_prob = off_team_data['FTP'].iloc[0] / 100
            point = 0
            last_shot_success = True
            for _ in range(2):
                random_value = np.random.uniform(0, 1)
                if random_value <= success_prob:
                    point += 1
                    last_shot_success = True
                else:
                    last_shot_success = False

            if not last_shot_success:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = point

    else:
        indicator_step2 = 'TOV'
        point = 0

    result = {
        'indicator_step1': indicator,
        'indicator_step2': indicator_step2,
        'point': point,
        'retry_occurred': retry_occurred
    }

    if nested_result is not None:
        result['nested_result'] = nested_result
        result['point'] += nested_result.get('point', 0)

    return result

In [13]:
simulation_function('ATL','BOS',0.8,100,Main_data)

{'indicator_step1': '2pt',
 'indicator_step2': 'MR',
 'point': 2,
 'retry_occurred': False}

### PACE

In [16]:
def game_simulation_function (OFF, DEF, rate, multiply, maindata):
    off_team_data = maindata[maindata['Team'] == OFF]
    def_team_data = maindata[maindata['Team'] == DEF]
    n=int((off_team_data['PACE'].iloc[0] + def_team_data['PACE'].iloc[0])/2)
    total_points = 0
    for _ in range(n):
        total_points += simulation_function(OFF, DEF, rate, multiply, maindata)['point']

    return total_points

In [18]:
def schedule_simulation(schedule, rate, multiply, maindata):
    wins = [] 
    loses = [] 

    for _, row in tqdm(schedule.iterrows(), total=len(schedule), desc="Simulating Games"):
        visitor = row['Visitor']
        home = row['Home']

        visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)

        home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

        while visitor_off_points == home_off_points:
            visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)
            home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

        if visitor_off_points > home_off_points:
            wins.append(visitor)
            loses.append(home)
        else:
            wins.append(home)
            loses.append(visitor)

    schedule['Win'] = wins
    schedule['Lose'] = loses
    return schedule


In [20]:
def schedule_simulation_basic(schedule, rate, multiply, maindata):
    wins = []  
    loses = []  

    for _, row in schedule.iterrows():
        visitor = row['Visitor']
        home = row['Home']

        visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)

        home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

        while visitor_off_points == home_off_points:
            visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)
            home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

        if visitor_off_points > home_off_points:
            wins.append(visitor)
            loses.append(home)
        else:
            wins.append(home)
            loses.append(visitor)

    schedule['Win'] = wins
    schedule['Lose'] = loses
    return schedule


In [22]:
def process_game(row, rate, multiply, maindata):
    visitor = row['Visitor']
    home = row['Home']

    visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)

    home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

    while visitor_off_points == home_off_points:
        visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)
        home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

    if visitor_off_points > home_off_points:
        return visitor, home
    else:
        return home, visitor

def schedule_simulation_parallel(schedule, rate, multiply, maindata, n_jobs=-1):
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_game)(row, rate, multiply, maindata) for _, row in tqdm(schedule.iterrows(), total=len(schedule), desc="Simulating Games")
    )

    wins, loses = zip(*results)
    schedule['Win'] = wins
    schedule['Lose'] = loses
    return schedule


In [24]:
def schedule_simulation_ntimes(n, schedule, rate, multiply, maindata):
    expanded_schedules = [] 

    for i in range(n):
        print(f"Simulation {i + 1}/{n}")
        simulated_schedule = schedule_simulation_parallel(schedule.copy(), rate, multiply, maindata)
        simulated_schedule['Simulation'] = i + 1 
        expanded_schedules.append(simulated_schedule)

    combined_schedule = pd.concat(expanded_schedules, ignore_index=True)
    return combined_schedule


In [26]:
def calculate_win_percentage(schedule, main_data):
    import pandas as pd

    win_counts = schedule['Win'].value_counts()  
    lose_counts = schedule['Lose'].value_counts() 

    game_counts = win_counts.add(lose_counts, fill_value=0) 

    win_percentage = (win_counts / game_counts).fillna(0)  


    win_percentage_table = pd.DataFrame({
        'Team': win_counts.index.union(lose_counts.index), 
        'Conference': win_counts.index.union(lose_counts.index).map(main_data.set_index('Team')['Conference']),  
        'Wins': win_counts,  
        'Losses': lose_counts,  
        'Win_Percentage': win_percentage  
    }).fillna(0) 

    win_percentage_table['Wins'] = win_percentage_table['Wins'].astype(int)
    win_percentage_table['Losses'] = win_percentage_table['Losses'].astype(int)

    win_percentage_table['Play_off'] = 0  
    win_percentage_table.loc[win_percentage_table.groupby('Conference')['Win_Percentage']
                             .rank(method='first', ascending=False) <= 10, 'Play_off'] = 1

    win_percentage_table = win_percentage_table.reset_index(drop=True)

    return win_percentage_table


In [28]:
from sklearn.metrics import mean_absolute_error

def calculate_mae(result_sample, origin_result):
    if isinstance(result_sample, pd.Series):
        result_sample = result_sample.reset_index()
        result_sample.columns = ['Team', 'Win_Percentage']

    if isinstance(origin_result, pd.Series):
        origin_result = origin_result.reset_index()
        origin_result.columns = ['Team', 'Win_Percentage']

    comparison_df = result_sample.merge(origin_result, on='Team', suffixes=('_Sample', '_Origin'))

    mae = mean_absolute_error(
        comparison_df['Win_Percentage_Sample'], 
        comparison_df['Win_Percentage_Origin']
    )
    return mae


In [30]:
def calculate_play_off_accuracy(predicted_result, origin_result):
    import pandas as pd

    merged = predicted_result[['Team', 'Play_off']].merge(origin_result[['Team', 'Play_off']], on='Team', suffixes=('_predicted', '_actual'))

    accuracy = (merged['Play_off_predicted'] == merged['Play_off_actual']).mean()

    return accuracy


In [54]:
import pandas as pd

team_comparison_list = []
results_acc = []
results_mae = []

for i in range(50):
    print(f"Running simulation {i+1}")
    exam_n = schedule_simulation_ntimes(1, Schedule, 0.98, 67, Main_data)

    sample = calculate_win_percentage(exam_n, Main_data)
    accuracy = calculate_play_off_accuracy(sample, Origin_result)
    mae = calculate_mae(sample, Origin_result)

    comparison = sample[['Team', 'Win_Percentage', 'Play_off']].merge(
        Origin_result[['Team', 'Win_Percentage', 'Play_off']], 
        on='Team', 
        suffixes=('_sim', '_origin')
    )
    
    comparison['MAE'] = abs(comparison['Win_Percentage_sim'] - comparison['Win_Percentage_origin'])
    comparison['Play_off_Correct'] = (comparison['Play_off_sim'] == comparison['Play_off_origin']).astype(int)

    comparison['Season'] = i + 1 

    team_comparison_list.append(comparison)
    results_acc.append(accuracy)
    results_mae.append(mae)

combined_comparison = pd.concat(team_comparison_list)

season_results = combined_comparison.groupby(['Season', 'Team']).agg({
    'Win_Percentage_sim': 'mean',
    'Play_off_sim': 'mean',
    'MAE': 'mean'
}).reset_index()

team_results = combined_comparison.groupby('Team').agg({
    'MAE': ['mean', 'std'],  
    'Play_off_Correct': ['mean', 'std'] 
}).reset_index()

team_results.columns = ['Team', 'MAE_mean', 'MAE_std', 'Play_off_Accuracy (%)', 'Play_off_Correct_std']
team_results['Play_off_Accuracy (%)'] *= 100  
team_results['Play_off_Correct_std'] *= 100  

average_acc = sum(results_acc) / len(results_acc)
average_mae = sum(results_mae) / len(results_mae)
std_acc = pd.Series(results_acc).std()
std_mae = pd.Series(results_mae).std()

season_results.to_excel("season_results_case2.xlsx", index=False)

print("Season results have been saved as 'season_results.xlsx'.")

print(f"Average Play-off Accuracy: {average_acc:.4f} ± {std_acc:.4f}")
print(f"Average MAE: {average_mae:.4f} ± {std_mae:.4f}")

print("\nTeam Results:")
print(team_results.to_string(index=False))


Running simulation 1
Simulation 1/1


Simulating Games:   0%|          | 0/1230 [00:00<?, ?it/s]

Running simulation 2
Simulation 1/1


Simulating Games:   0%|          | 0/1230 [00:00<?, ?it/s]

Running simulation 3
Simulation 1/1


...

Running simulation 49
Simulation 1/1


Simulating Games:   0%|          | 0/1230 [00:00<?, ?it/s]

Running simulation 50
Simulation 1/1


Simulating Games:   0%|          | 0/1230 [00:00<?, ?it/s]

Season results have been saved as 'season_results.xlsx'.
Average Play-off Accuracy: 0.8040 ± 0.0455
Average MAE: 0.0915 ± 0.0074

Team Results:
Team  MAE_mean  MAE_std  Play_off_Accuracy (%)  Play_off_Correct_std
 ATL  0.115633 0.060914                   96.0             19.794866
 BKN  0.107561 0.055037                   16.0             37.032804
 BOS  0.071815 0.042641                  100.0              0.000000
 CHA  0.105220 0.055852                   86.0             35.050983
 CHI  0.049134 0.033093                   66.0             47.851812
 CLE  0.089907 0.049160                   86.0             35.050983
 DAL  0.055746 0.037661                   90.0             30.304576
 DEN  0.180366 0.055695                   74.0             44.308750
 DET  0.169732 0.048619                   94.0             23.989794
 GSW  0.061208 0.047288                   98.0             14.142136
 HOU  0.056098 0.033238                   10.0             30.304576
 IND  0.047900 0.036692     